# 模型下載

In [1]:
import time
from os import times

from transformers import AutoProcessor, Gemma3ForConditionalGeneration
import torch
import os

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.cuda.get_device_name()

'NVIDIA RTX PRO 6000 Blackwell Workstation Edition'

In [2]:
model_id = "google/gemma-3-12b-pt"

model = Gemma3ForConditionalGeneration.from_pretrained(
    model_id, device_map="auto"
).eval()

processor = AutoProcessor.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/torch/cuda/__init__.py:829: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
Loading checkpoint shards: 100%|██████████| 5/5 [00:04<00:00,  1.22it/s]
Some parameters are on the meta device because they were offloaded to the cpu and disk.
Fetching 2 files: 100%|██████████| 2/2 [00:00<00:00, 53430.62it/s]
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [4]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [5]:
torch.cuda.empty_cache()

In [3]:
model

Gemma3ForConditionalGeneration(
  (model): Gemma3Model(
    (vision_tower): SiglipVisionModel(
      (vision_model): SiglipVisionTransformer(
        (embeddings): SiglipVisionEmbeddings(
          (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
          (position_embedding): Embedding(4096, 1152)
        )
        (encoder): SiglipEncoder(
          (layers): ModuleList(
            (0-26): 27 x SiglipEncoderLayer(
              (layer_norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
              (self_attn): SiglipAttention(
                (k_proj): Linear(in_features=1152, out_features=1152, bias=True)
                (v_proj): Linear(in_features=1152, out_features=1152, bias=True)
                (q_proj): Linear(in_features=1152, out_features=1152, bias=True)
                (out_proj): Linear(in_features=1152, out_features=1152, bias=True)
              )
              (layer_norm2): LayerNorm((1152,), eps=1e-06, elementwi

# 準備資料

In [6]:
import pandas as pd
from PIL import Image
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

In [7]:
file_locate = "/tmp/pycharm_project_159/" #遠端環境

images = os.listdir(file_locate + "/dataset/擊球數據整理/images")
Inputs = pd.read_csv(file_locate + "/dataset/擊球數據整理/question_Input.csv")

init_prompt = open(file_locate + "/dataset/init_prompt.txt").read()
rule = pd.read_excel(file_locate + "/dataset/回饋規則.xlsx")

In [8]:
rule

,球路類型,結果,原因,改善建議
0,Pull左飛球,失誤,上桿(P2~3)時，角度過於陡峭,上桿(P2~3)時，肩膀往右轉動
1,Pull左飛球,失誤,桿頭頂點過高,肩膀往右轉動
2,Pull左飛球,失誤,下桿角度過於陡峭，左手腕過度外展，肩關節伸展抬起,下桿時，左手臂打直、左手腕維持固定、肩膀自然旋轉
3,Pull左飛球,失誤,桿面關閉，擊球點位於球的外側,擊球時，左手腕維持固定，注意擊球點位置
4,Pull Hook左拉左曲球,失誤,上桿(P2~3)時，角度過於陡峭,上桿(P2~3)時，肩膀往右轉動
5,Pull Hook左拉左曲球,失誤,桿頭頂點過高,肩膀請往右轉動
6,Pull Hook左拉左曲球,失誤,下桿角度過於陡峭，手腕過度彎曲，過度由內而外的路徑,下桿時，左手臂打直、左手腕維持固定
7,Pull Hook左拉左曲球,失誤,桿面關閉，擊球點位於球的外側，手腕繼續彎曲未保持向前,擊球時，左手腕應恢復成未彎曲的狀態，及注意擊球點位置
8,Pull Slice 左拉右曲球,失誤,通常是因為上桿時P2過於內側，手臂和身體過於靠近卡住之後，反而在下桿時由外側下桿、或是軸心偏...,上桿時維持軸心，P2時桿身平行於雙腳之平行線，保持桿面角度
9,Pull Slice 左拉右曲球,失誤,擺動路徑過於內向,胸椎充分旋轉，重心往左


In [8]:
import base64

def encode_base64(image):
    with open(image, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")

In [9]:
class GolfDataset(Dataset):
    def __init__(self, Input):
        self.num = Input["num"]
        self.images = []
        self.questions = Input["Input"]
        self.ground_truth = Input["GroundTruth"]
    def __len__(self):
        return len(self.questions)
    def __getitem__(self, idx):
        num = self.num.iloc[idx]
        question = self.questions.iloc[idx]
        image = encode_base64(file_locate + "/dataset/擊球數據整理/images/"+"combined_" +str(self.num.iloc[idx]) + ".jpg")
        ground_truth = self.ground_truth.iloc[idx]
        return num,image,question,ground_truth

In [10]:
golf_dataset = GolfDataset(Inputs)
golf_dataloader = DataLoader(golf_dataset, shuffle=False)

In [29]:
result_df = pd.DataFrame()


rule = str(rule)
for num ,images , questions, ground_truth in golf_dataloader:
    rule = str(rule)
    images = str(images)
    messages = [
        {
            "role": "system",
            "content": [{"type": "text", "text": init_prompt+"以下是回饋規則"+rule}],
        },
        {
          "role": "user",
          "content": [
                {"type": "image", "base64": images},
                {"type": "text", "text": questions},
            ],
        },
    ]
    inputs = processor.apply_chat_template(
        messages, add_generation_prompt=True, tokenize=True,
        return_dict=True, return_tensors="pt"
    ).to(model.device, dtype=torch.bfloat16)
    
    input_len = inputs["input_ids"].shape[-1]
    
    with torch.inference_mode():
        generation = model.generate(
            **inputs,
            max_new_tokens=1024,
            do_sample=True,
            temperature=0.1
        )
        generation = generation[0][input_len:]
    
    decoded = processor.decode(generation, skip_special_tokens=True)
    result_df = pd.concat([result_df, pd.DataFrame({"num":num, "answer":decoded})])
    
    result_df["answer"] = result_df["answer"].str.replace(" ", "")  
    print(decoded)
    

好的，我將根據您提供的資訊進行分析，並提供詳細的口語化動作建議。

**分析結果：**

綜合觀察圖片和擊球數據，學員在P5~6下桿時，存在明顯的左拉左曲球問題。從正面和側面圖片觀察，發現學員在下桿時，左手腕過度外展，肩關節伸展抬起，導致桿面關閉，擊球點位於球的外側。球速和飛行距離數據顯示，球速為66.43，發射角度為7.198，發射方向為-6.172，飛行距離為179，ClubAngleFace為-15，ClubAnglePath為6.901，這些數據都指向左拉左曲球的傾向。

**整合分析建議：**

“您好，根據我們全面的揮杆分析，您的揮杆動作與教練大致相同，請繼續保持。然而您的揮杆有幾個地方可以加強。在您的P5~6下桿階段，考慮調整您的左手腕，試著讓它保持相對固定，不要過度外展，同時注意肩膀的自然旋轉，不要過度伸展。這將有助於改善您的擊球路徑，減少左拉左曲球的發生。另外，在下桿時，請嘗試利用臀部發力，而不是過早地向前傾身，這樣可以更好地協調身體的旋轉，保持揮桿的穩定性。”

**具體調整建議：**

“首先，讓我們從P5~6的下桿開始。當您準備擊球時，請意識到您的左手腕應該保持相對固定，不要過度彎曲或外展。想像一下，您的左手腕就像一個支撐點，幫助您保持揮桿的穩定性。同時，請注意您的肩膀，不要過度伸展，而是讓它們自然地旋轉。這就像一個轉盤，在旋轉過程中，您的手臂和身體會自然地跟著轉動。此外，在擊球時，請嘗試利用臀部發力，而不是過早地向前傾身。這樣可以更好地協調身體的旋轉，保持揮桿的穩定性，並減少左拉左曲球的發生。您可以想像一下，您的臀部就像一個動力源，在旋轉過程中，它會提供力量，幫助您擊出更正的球路。”

**最終分析：**

```json
{
        "球路": "Pull Hook 左拉左曲球",
         "原因": "下桿時，左手腕過度外展，肩關節伸展抬起，導致桿面關閉，擊球點位於球的外側。",
         "建議": "在P5~6下桿時，請嘗試讓左手腕保持相對固定，不要過度外展，同時注意肩膀的自然旋轉，不要過度伸展。此外，在擊球時，請嘗試利用臀部發力，而不是過早地向前傾身，這將有助於改善您的擊球路徑，減少左拉左曲球的發生。"
}
```

好的，我將根據您提供的資訊進行分析，並提供詳細的口語化動作建議。

**分析結果：**


In [15]:
result_df

,num,answer
0,186382,"```json\n{\n""球路"":""左拉左曲球"",\n""原因"":""從圖片分析來看，在P2的桿..."
0,186387,"```json\n{\n""球路"":""小右曲球"",\n""原因"":""從圖片分析來看，在P2的擊球..."
0,186410,"```json\n{\n""球路"":""PushSlice右拉右曲球"",\n""原因"":""從圖片分..."
0,186416,"```json\n{\n""球路"":""左拉左曲球"",\n""原因"":""從正面圖片分析，學員在P2..."
0,198514,"```json\n{\n""球路"":""PushSlice右拉右曲球"",\n""原因"":""從正面和..."
0,199353,"```json\n{\n""球路"":""PushSlice右拉右曲球"",\n""原因"":""從圖片分..."


In [16]:
result_df.to_csv(file_locate+"/experiment_result/"+"test_"+model_id+"_output_result.csv", index=False, encoding="utf-8")